# 機械学習に関するテクニック
機械学習でよく使うテクニックをまとめる。

In [1]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder
# # import statsmodels.api as sma
from sklearn.model_selection import train_test_split ,cross_val_score, KFold, RepeatedKFold,StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer,KNNImputer
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
# from sklearn import tree
# from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,AdaBoostClassifier, GradientBoostingRegressor, GradientBoostingClassifier


%matplotlib inline
import matplotlib.pyplot as plt


## ○ Polars

### - 欠損値を含むカラムの一覧を取得する

In [3]:
# データセットのカラム名やデータ型は分かっているので、dtypesを指定しておく。
dtypes = {
    "species": str,
    'island': str,
    'culmen_length_mm': pl.Float32, # くちばしの長さ[mm]
    'culmen_depth_mm': pl.Float32, # くちばしの高さ[mm]
    'flipper_length_mm': pl.Float32, # 翼の長さ[mm]
    'body_mass_g': pl.Float32, # 体重[g]
    'sex': str
}

# ペンギンのデータセット読み込み。欠損値がNAとして含まれているので、null_values="NA"を指定しないと読み込みエラーになる。
df = pl.read_csv('../Python/sample_data/ML_sample/penguins_size.csv',dtypes=dtypes, null_values='NA')

#### 方法１

In [28]:
(df
 .null_count().transpose(include_header=True)
 .filter(pl.col('column_0') > 0)
 .get_column('column').to_list()
)

['culmen_length_mm',
 'culmen_depth_mm',
 'flipper_length_mm',
 'body_mass_g',
 'sex']

上記ではnull_countの結果をカラム名（ヘッダー）を含めて転置して、  
転置後の欠損値数(column_0)でフィルターし、その時のヘッダーを返している。

#### 方法2

In [5]:
# 欠損値を含むカラムの列インデックスを取得
null_counts = df.null_count().to_numpy().reshape(7)
# np.whereで欠損値が０以上のカラムインデックスを取得。np.whereの結果はタプルで返ってくるので注意。
null_contain_cols_idx = np.where(null_counts > 0)[0]

In [6]:
# インデックスに対応するカラム名を取得
null_contain_cols = np.array(df.columns)[null_contain_cols_idx]

少々煩雑だが、numpyでもできる。

### - 複数カラムを対象として欠損値を含む行を抽出する
EDAの中で欠損値対応前後の値を見たいときがあるので、Polarsでもインデックス付きで行を抽出しておくと便利。  
複数カラムに対してフィルタリングするためにExprをfor文で生成する必要があるので、若干めんどくさい。  
他に良い方法がないか模索中。

In [33]:
df.null_count()

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
u32,u32,u32,u32,u32,u32,u32
0,0,2,2,2,2,10


In [31]:
# 複数のカラムに対して、欠損値がある行をフィルタリングするためのExprを生成する
expr = pl

for i, col in enumerate(null_contain_cols):
    if i == 0:
        expr = expr.col(col).is_null()
    else:
        # __or__は演算子'|'と同じ
        expr = expr.__or__(pl.col(col).is_null())

In [ ]:
expr

In [34]:
# 欠損値がある行のindexを取得
null_index = df.with_row_count().filter(expr).get_column('row_nr')

In [35]:
null_index

row_nr
u32
3
8
9
10
11
47
246
286
324


In [36]:
df.with_row_count().filter(
    pl.col('sex').is_null()
)

row_nr,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
u32,str,str,f32,f32,f32,f32,str
3,"""Adelie""","""Torgersen""",null,null,null,null,null
8,"""Adelie""","""Torgersen""",34.099998,18.1,193.0,3475.0,null
9,"""Adelie""","""Torgersen""",42.0,20.200001,190.0,4250.0,null
10,"""Adelie""","""Torgersen""",37.799999,17.1,186.0,3300.0,null
11,"""Adelie""","""Torgersen""",37.799999,17.299999,180.0,3700.0,null
47,"""Adelie""","""Dream""",37.5,18.9,179.0,2975.0,null
246,"""Gentoo""","""Biscoe""",44.5,14.3,216.0,4100.0,null
286,"""Gentoo""","""Biscoe""",46.200001,14.4,214.0,4650.0,null
324,"""Gentoo""","""Biscoe""",47.299999,13.8,216.0,4725.0,null


確かに取得できていそう。